# Convolutional Neural Network

### Importing the libraries

In [27]:
!pip install --upgrade tensorflow keras Pillow

^C


You should consider upgrading via the 'c:\users\samue\área de trabalho\unbe\disciplinas\deep\deep_venv\scripts\python.exe -m pip install --upgrade pip' command.


In [28]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [29]:
tf.__version__

'2.6.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [30]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [31]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [32]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [33]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [34]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [35]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [36]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='tanh'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [37]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [38]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [39]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [40]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [41]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 28s 111ms/step - loss: 0.6689 - accuracy: 0.5880 - val_loss: 0.6305 - val_accuracy: 0.6545
Epoch 2/25
250/250 [==============================] - 28s 112ms/step - loss: 0.5897 - accuracy: 0.6865 - val_loss: 0.5428 - val_accuracy: 0.7320
Epoch 3/25
250/250 [==============================] - 27s 108ms/step - loss: 0.5439 - accuracy: 0.7260 - val_loss: 0.5019 - val_accuracy: 0.7660
Epoch 4/25
250/250 [==============================] - 26s 103ms/step - loss: 0.5091 - accuracy: 0.7459 - val_loss: 0.4909 - val_accuracy: 0.7680
Epoch 5/25
250/250 [==============================] - 31s 125ms/step - loss: 0.4796 - accuracy: 0.7701 - val_loss: 0.4902 - val_accuracy: 0.7720
Epoch 6/25
250/250 [==============================] - 31s 126ms/step - loss: 0.4631 - accuracy: 0.7811 - val_loss: 0.4620 - val_accuracy: 0.7850
Epoch 7/25
250/250 [==============================] - 31s 123ms/step - loss: 0.4487 - accuracy: 0.7857 - val_loss: 0.4473 - val_ac

## Part 4 - Making a single prediction

In [50]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/dog.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [51]:
print(prediction)

1 -> dog
2 -> cat
3 -> dog
4 -> dog
5 -> dog
6 -> cat
7 -> cat
8 -> dog
9 -> dog
10 -> dog
11 -> dog
12 -> cat


## Fim